In [1]:
import pandas as pd
import numpy as np

# Fixed random seed to ensure reproducibility of generated data
SEED = 20251217
rng = np.random.default_rng(SEED)

# Raw dataset downloaded from Kaggle (Amazon_Sale_Report.csv)
# https://www.kaggle.com/datasets/mukundsavaliya/e-comm-dataset
df_raw = pd.read_csv("Amazon_Sale_Report.csv", low_memory=False)

# Select required source columns (whitelist)
keep_cols = [
    "Order ID",        # Source order identifier (string)
    "Date",            # Order date (string)
    "SKU",             # Product business key (string)
    "Qty",             # Order item quantity
    "Fulfilment",      # Used to derive merchant (Amazon / Merchant)
    "Category",        # Used to derive product name
    "ship-city",       # Used to derive client and product name
    "ship-state"       # Used to derive client
]

df = df_raw[keep_cols].copy()

# Normalize column names for further processing
df = df.rename(columns={
    "Order ID": "order_id_src",
    "Date": "order_date_src",
    "SKU": "sku",
    "Qty": "quantity",
    "Fulfilment": "fulfilment",
    "Category": "category",
    "ship-city": "ship_city",
    "ship-state": "ship_state",
})

# Each SKU should correspond to exactly one product name.
# If a SKU appears with multiple categories, the most frequent
# category is selected.
sku_top_category = (
    df.groupby(["sku", "category"])
      .size()
      .reset_index(name="cnt")
      .sort_values(["sku", "cnt"], ascending=[True, False])
      .drop_duplicates("sku")[["sku", "category"]]
)

# Generate a stable random uint16 value per SKU
# (Randomness is fixed by the global SEED)
sku_top_category["rand_u16"] = rng.integers(
    low=0, high=65536,
    size=len(sku_top_category),
    dtype=np.uint16
)

# Construct product name: <category>_<random_uint16>
sku_top_category["productname"] = (
    sku_top_category["category"].astype(str) + "_" +
    sku_top_category["rand_u16"].astype(str)
)

# Attach productname back to the transactional dataset
df = df.merge(
    sku_top_category[["sku", "productname"]],
    on="sku",
    how="left"
)

# Client is defined as a unique (city, state) pair.
df["client_key"] = (
    df["ship_city"].fillna("UNKNOWN_CITY") + "|" +
    df["ship_state"].fillna("UNKNOWN_STATE")
)

# Build client dimension table (distinct clients)
client_dim = (
    df[["client_key", "ship_city", "ship_state"]]
    .drop_duplicates()
    .copy()
)

# Generate two stable random uint8 numbers per client
client_dim["rand_w"] = rng.integers(
    low=0, high=256,
    size=len(client_dim),
    dtype=np.uint8
)
client_dim["rand_n"] = rng.integers(
    low=0, high=256,
    size=len(client_dim),
    dtype=np.uint8
)

# Construct synthetic contact address
# Format: "<W> W <N> N St. <City> <State>"
client_dim["contact_address"] = (
    client_dim["rand_w"].astype(str) + " W " +
    client_dim["rand_n"].astype(str) + " N " +
    "St. " +
    client_dim["ship_city"].fillna("UNKNOWN_CITY").astype(str) + " " +
    client_dim["ship_state"].fillna("UNKNOWN_STATE").astype(str)
)

# Assign stable integer clientId (surrogate key)
client_dim = client_dim.sort_values("client_key").reset_index(drop=True)
client_dim["clientId"] = np.arange(
    start=1,
    stop=len(client_dim) + 1,
    dtype=np.int32
)

# Attach clientId and contact_address to fact-level data
df = df.merge(
    client_dim[["client_key", "clientId", "contact_address"]],
    on="client_key",
    how="left"
)

# All quantities are increased by 1 to avoid zero-quantity order items
df["quantity"] = df["quantity"] + 1

# Generate productId (sku -> int mapping)
product_key = (
    df[["sku"]]
    .drop_duplicates()
    .sort_values("sku")
    .reset_index(drop=True)
)

product_key["productId"] = np.arange(
    start=1,
    stop=len(product_key) + 1,
    dtype=np.int32
)

# Attach productId back to main dataframe
df = df.merge(product_key, on="sku", how="left")

# Generate orderId (order_id_src -> int mapping)
order_key = (
    df[["order_id_src"]]
    .drop_duplicates()
    .sort_values("order_id_src")
    .reset_index(drop=True)
)

order_key["orderId"] = np.arange(
    start=1,
    stop=len(order_key) + 1,
    dtype=np.int32
)

# Attach orderId back to main dataframe
df = df.merge(order_key, on="order_id_src", how="left")

# Sanity checks
assert df["sku"].nunique() == df["productId"].nunique()
assert df["order_id_src"].nunique() == df["orderId"].nunique()
assert (df["quantity"] > 0).all()

print("After ID mapping:")
print("Products:", df["productId"].nunique())
print("Orders:", df["orderId"].nunique())
print("Order items:", len(df))

df.head()

After ID mapping:
Products: 7195
Orders: 120378
Order items: 128975


,order_id_src,order_date_src,sku,quantity,fulfilment,category,ship_city,ship_state,productname,client_key,clientId,contact_address,productId,orderId
0,405-8078784-5731545,04-30-22,SET389-KR-NP-S,1,Merchant,Set,MUMBAI,MAHARASHTRA,Set_87,MUMBAI|MAHARASHTRA,4931,237 W 134 N St. MUMBAI MAHARASHTRA,6993,70509
1,171-9198151-1101146,04-30-22,JNE3781-KR-XXXL,2,Merchant,kurta,BENGALURU,KARNATAKA,kurta_32469,BENGALURU|KARNATAKA,817,129 W 211 N St. BENGALURU KARNATAKA,4437,13616
2,404-0687676-7273146,04-30-22,JNE3371-KR-XL,2,Amazon,kurta,NAVI MUMBAI,MAHARASHTRA,kurta_18732,NAVI MUMBAI|MAHARASHTRA,5461,137 W 13 N St. NAVI MUMBAI MAHARASHTRA,2634,45302
3,403-9615377-8133951,04-30-22,J0341-DR-L,1,Merchant,Western Dress,PUDUCHERRY,PUDUCHERRY,Western Dress_10103,PUDUCHERRY|PUDUCHERRY,6269,104 W 175 N St. PUDUCHERRY PUDUCHERRY,1768,43683
4,407-1069790-7240320,04-30-22,JNE3671-TU-XXXL,2,Amazon,Top,CHENNAI,TAMIL NADU,Top_29033,CHENNAI|TAMIL NADU,1516,111 W 170 N St. CHENNAI TAMIL NADU,3838,90100


In [2]:
print("\n=== Unique counts per column ===")
for col in df.columns:
    print(f"{col:20s}: {df[col].nunique()}")


=== Unique counts per column ===
order_id_src        : 120378
order_date_src      : 91
sku                 : 7195
quantity            : 10
fulfilment          : 2
category            : 9
ship_city           : 8955
ship_state          : 69
productname         : 7074
client_key          : 9149
clientId            : 9149
contact_address     : 9149
productId           : 7195
orderId             : 120378


In [3]:
dominant_fulfilment = (
    df.groupby(["productId", "fulfilment"])
      .size()
      .reset_index(name="cnt")
      .sort_values(["productId", "cnt"], ascending=[True, False])
      .drop_duplicates("productId")[["productId", "fulfilment"]]
)

# Build base product dimension (one row per product)
stg_product = (
    dominant_fulfilment
    .merge(
        df[["productId", "productname"]].drop_duplicates(),
        on="productId",
        how="left"
    )
    .sort_values("productId")
    .reset_index(drop=True)
)

# Derive merchant_id from fulfilment
# Amazon -> 1, Merchant -> 2
stg_product["merchant_id"] = stg_product["fulfilment"].map({
    "Amazon": 1,
    "Merchant": 2
}).astype(np.int32)

# Generate synthetic product price
stg_product["price"] = (
    rng.uniform(50.0, 500.0, size=len(stg_product))
    .round(2)
)

# Generate synthetic stock quantity
stg_product["stock"] = rng.integers(
    low=0,
    high=1001,
    size=len(stg_product),
    dtype=np.int32
)

# Generate product description
stg_product["description"] = (
    "Product " + stg_product["productname"].astype(str)
)

# stg_product structure
stg_product = stg_product[[
    "productId",
    "productname",
    "price",
    "stock",
    "description",
    "merchant_id"
]]

In [4]:
print("stg_product shape:", stg_product.shape)
print(stg_product.head())

# Hard checks
assert stg_product["productId"].is_unique
assert stg_product["productname"].notnull().all()
assert stg_product["merchant_id"].isin([1, 2]).all()

stg_product shape: (7195, 6)
   productId   productname   price  stock           description  merchant_id
0          1  Bottom_46599  449.61    210  Product Bottom_46599            1
1          2  Bottom_33142  389.54    864  Product Bottom_33142            1
2          3  Bottom_52634  353.43    595  Product Bottom_52634            2
3          4  Bottom_36604  476.39    278  Product Bottom_36604            2
4          5  Bottom_27287  426.83    883  Product Bottom_27287            1


In [5]:
# Build stg_merchant (Amazon / Merchant)
stg_merchant = pd.DataFrame({
    "merchantId": [1, 2],
    "storename": ["Amazon", "Merchant"],
})

# Synthetic attributes (deterministic via the same rng)
stg_merchant["description"] = "Generated merchant"
stg_merchant["email"] = stg_merchant["storename"].str.lower() + "@example.com"
stg_merchant["contact_address"] = stg_merchant["storename"] + " HQ"

# Profit: synthetic (since we are not generating amount)
stg_merchant["profit"] = rng.uniform(1_000_000.0, 5_000_000.0, size=len(stg_merchant)).round(2)

print("stg_merchant shape:", stg_merchant.shape)
print(stg_merchant)

stg_merchant shape: (2, 6)
   merchantId storename         description                 email  \
0           1    Amazon  Generated merchant    amazon@example.com   
1           2  Merchant  Generated merchant  merchant@example.com   

  contact_address      profit  
0       Amazon HQ  4459452.66  
1     Merchant HQ  1814097.34  


In [6]:
# Build stg_client (one row per clientId)
stg_client = (
    client_dim[["clientId", "contact_address"]]
    .drop_duplicates()
    .sort_values("clientId")
    .reset_index(drop=True)
)

# Generate synthetic username/email
stg_client["username"] = "user_" + stg_client["clientId"].astype(str)
stg_client["email"] = "user_" + stg_client["clientId"].astype(str) + "@example.com"

# Reorder columns to match the staging table schema
stg_client = stg_client[["clientId", "username", "email", "contact_address"]]

# Sanity checks
assert stg_client["clientId"].is_unique
assert stg_client["contact_address"].notnull().all()

print("stg_client shape:", stg_client.shape)
print(stg_client.head())

stg_client shape: (9149, 4)
   clientId username               email  \
0         1   user_1  user_1@example.com   
1         2   user_2  user_2@example.com   
2         3   user_3  user_3@example.com   
3         4   user_4  user_4@example.com   
4         5   user_5  user_5@example.com   

                                     contact_address  
0  203 W 124 N St. (Chikmagalur disterict).     (...  
1  53 W 230 N St. (Via Cuncolim)Quepem,South Goa GOA  
2               244 W 203 N St. ,HYDERABAD TELANGANA  
3  67 W 240 N St. ,raibarely road faizabad (Ayodh...  
4             45 W 175 N St. ..katra JAMMU & KASHMIR  


In [7]:
# Build stg_orders (one row per orderId)
stg_orders = (
    df.groupby("orderId", as_index=False)
      .agg(
          client_id=("clientId", "first"),
          order_date_src=("order_date_src", "first")
      )
      .sort_values("orderId")
      .reset_index(drop=True)
)

# Parse order_date_src into datetime
# Source looks like '04-30-22' -> mm-dd-yy
stg_orders["order_date"] = pd.to_datetime(
    stg_orders["order_date_src"],
    format="%m-%d-%y",
    errors="raise"
)

# Final schema: (orderId, client_id, order_date)
stg_orders = stg_orders[["orderId", "client_id", "order_date"]]

# Sanity checks
assert stg_orders["orderId"].is_unique
assert stg_orders["client_id"].notnull().all()
assert stg_orders["client_id"].between(1, stg_client["clientId"].max()).all()

print("stg_orders shape:", stg_orders.shape)
print(stg_orders.head())

stg_orders shape: (120378, 3)
   orderId  client_id order_date
0        1       6202 2022-06-07
1        2       3063 2022-06-09
2        3       3772 2022-06-07
3        4       2643 2022-05-04
4        5       2471 2022-04-03


In [8]:
# Build stg_order_item (one row per (order_id, product_id))
stg_order_item = (
    df.groupby(["orderId", "productId"], as_index=False)
      .agg(quantity=("quantity", "sum"))
      .rename(columns={
          "orderId": "order_id",
          "productId": "product_id"
      })
      .sort_values(["order_id", "product_id"])
      .reset_index(drop=True)
)

# Sanity checks
assert (stg_order_item["quantity"] > 0).all()
assert not stg_order_item.duplicated(subset=["order_id", "product_id"]).any()

print("stg_order_item shape:", stg_order_item.shape)
print(stg_order_item.head())

stg_order_item shape: (128968, 3)
   order_id  product_id  quantity
0         1        2217         2
1         2         684         2
2         3        2904         2
3         4        6444         2
4         5        2904         2


In [9]:
def print_table_info(name, df):
    print(f"\n=== {name} ===")
    print(f"Shape: {df.shape[0]} rows × {df.shape[1]} columns")
    print("Columns:")
    for col in df.columns:
        print(f"  - {col}")

print_table_info("stg_product", stg_product)
print_table_info("stg_merchant", stg_merchant)
print_table_info("stg_client", stg_client)
print_table_info("stg_orders", stg_orders)
print_table_info("stg_order_item", stg_order_item)


=== stg_product ===
Shape: 7195 rows × 6 columns
Columns:
  - productId
  - productname
  - price
  - stock
  - description
  - merchant_id

=== stg_merchant ===
Shape: 2 rows × 6 columns
Columns:
  - merchantId
  - storename
  - description
  - email
  - contact_address
  - profit

=== stg_client ===
Shape: 9149 rows × 4 columns
Columns:
  - clientId
  - username
  - email
  - contact_address

=== stg_orders ===
Shape: 120378 rows × 3 columns
Columns:
  - orderId
  - client_id
  - order_date

=== stg_order_item ===
Shape: 128968 rows × 3 columns
Columns:
  - order_id
  - product_id
  - quantity


In [10]:
stg_product.to_csv("stg_tables/stg_product.csv", index=False, encoding="utf-8")
stg_merchant.to_csv("stg_tables/stg_merchant.csv", index=False, encoding="utf-8")
stg_client.to_csv("stg_tables/stg_client.csv", index=False, encoding="utf-8")
stg_orders.to_csv("stg_tables/stg_orders.csv", index=False, encoding="utf-8")
stg_order_item.to_csv("stg_tables/stg_order_item.csv", index=False, encoding="utf-8")